# Python Lab 4: QPSK/QAM Simulation with EVM Analysis

**EE 451: Communications Systems**

**Name:** _________________________ **Date:** _____________

---

## Objectives

1. Generate QPSK and QAM constellation diagrams
2. Simulate I/Q modulation and demodulation
3. Add noise and observe constellation spreading
4. Calculate Error Vector Magnitude (EVM)
5. Analyze the relationship between EVM and SNR

---

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

plt.rcParams['figure.figsize'] = (10, 8)
print("Setup complete!")

## Part 1: QPSK Constellation (20 points)

QPSK encodes 2 bits per symbol using 4 phase states.

### Task 1.1: Generate QPSK Symbols

In [ ]:
# QPSK constellation points (Gray coded)
# 00 -> +1+j, 01 -> -1+j, 11 -> -1-j, 10 -> +1-j
qpsk_map = {
    (0, 0): (1 + 1j) / np.sqrt(2),
    (0, 1): (-1 + 1j) / np.sqrt(2),
    (1, 1): (-1 - 1j) / np.sqrt(2),
    (1, 0): (1 - 1j) / np.sqrt(2)
}

# Generate random bits
np.random.seed(42)
num_symbols = 1000
bits = np.random.randint(0, 2, num_symbols * 2)

# TODO: Map bits to QPSK symbols
symbols = []
for i in range(0, len(bits), 2):
    bit_pair = (bits[i], bits[i+1])
    symbols.append(qpsk_map[bit_pair])
symbols = np.array(symbols)

# Plot constellation
plt.figure(figsize=(8, 8))
plt.scatter(symbols.real, symbols.imag, alpha=0.5, s=10)

# Mark ideal constellation points
for bits, point in qpsk_map.items():
    plt.plot(point.real, point.imag, 'r*', markersize=20)
    plt.annotate(f'{bits[0]}{bits[1]}', (point.real, point.imag), 
                 textcoords='offset points', xytext=(10, 10), fontsize=12)

plt.xlabel('In-Phase (I)')
plt.ylabel('Quadrature (Q)')
plt.title('QPSK Constellation (No Noise)')
plt.grid(True, alpha=0.3)
plt.axis('equal')
plt.xlim(-1.5, 1.5)
plt.ylim(-1.5, 1.5)
plt.axhline(y=0, color='k', linewidth=0.5)
plt.axvline(x=0, color='k', linewidth=0.5)
plt.show()

print(f"Number of symbols: {len(symbols)}")
print(f"Bits per symbol: 2")
print(f"Average symbol power: {np.mean(np.abs(symbols)**2):.3f}")

### Question 1.1 (10 points)

a) Why is QPSK normalized by $1/\sqrt{2}$?

b) What is Gray coding and why is it used in QPSK?

**Your Answer:**

*[Write your answer here]*

## Part 2: 16-QAM Constellation (20 points)

### Task 2.1: Generate 16-QAM Symbols

In [ ]:
# TODO: Create 16-QAM constellation
# 16-QAM uses 4x4 grid with levels: -3, -1, +1, +3
levels = np.array([-3, -1, 1, 3])

# Create all 16 constellation points
qam16_points = []
for i in levels:
    for q in levels:
        qam16_points.append(i + 1j*q)
qam16_points = np.array(qam16_points)

# Normalize to unit average power
avg_power = np.mean(np.abs(qam16_points)**2)
qam16_points = qam16_points / np.sqrt(avg_power)

# Generate random 16-QAM symbols
num_symbols = 1000
symbol_indices = np.random.randint(0, 16, num_symbols)
symbols_16qam = qam16_points[symbol_indices]

# Plot
plt.figure(figsize=(8, 8))
plt.scatter(symbols_16qam.real, symbols_16qam.imag, alpha=0.5, s=10)

# Mark ideal points
for point in qam16_points:
    plt.plot(point.real, point.imag, 'r*', markersize=15)

plt.xlabel('In-Phase (I)')
plt.ylabel('Quadrature (Q)')
plt.title('16-QAM Constellation (No Noise)')
plt.grid(True, alpha=0.3)
plt.axis('equal')
plt.axhline(y=0, color='k', linewidth=0.5)
plt.axvline(x=0, color='k', linewidth=0.5)
plt.show()

print(f"Number of constellation points: 16")
print(f"Bits per symbol: 4")
print(f"Average symbol power: {np.mean(np.abs(symbols_16qam)**2):.3f}")

### Question 2.1 (10 points)

a) How many bits are encoded in each 16-QAM symbol?

b) Compare the minimum distance between constellation points for QPSK vs 16-QAM (at same average power). Which is more robust to noise?

**Your Answer:**

*[Write your answer here]*

## Part 3: Adding Noise and Observing Effects (25 points)

### Task 3.1: Add AWGN and Compare Constellations

In [ ]:
def add_awgn(symbols, snr_db):
    """Add AWGN to symbols at specified SNR."""
    signal_power = np.mean(np.abs(symbols)**2)
    snr_linear = 10**(snr_db / 10)
    noise_power = signal_power / snr_linear
    
    # Complex noise (I and Q components)
    noise = np.sqrt(noise_power/2) * (np.random.randn(len(symbols)) + 
                                       1j * np.random.randn(len(symbols)))
    return symbols + noise

# TODO: Test different SNR values
snr_values = [20, 15, 10, 5]  # dB

fig, axes = plt.subplots(2, 4, figsize=(16, 8))

for i, snr in enumerate(snr_values):
    # QPSK with noise
    noisy_qpsk = add_awgn(symbols, snr)
    axes[0, i].scatter(noisy_qpsk.real, noisy_qpsk.imag, alpha=0.3, s=5)
    for point in qpsk_map.values():
        axes[0, i].plot(point.real, point.imag, 'r*', markersize=15)
    axes[0, i].set_title(f'QPSK, SNR = {snr} dB')
    axes[0, i].set_xlim(-2, 2)
    axes[0, i].set_ylim(-2, 2)
    axes[0, i].grid(True, alpha=0.3)
    axes[0, i].set_aspect('equal')
    
    # 16-QAM with noise
    noisy_16qam = add_awgn(symbols_16qam, snr)
    axes[1, i].scatter(noisy_16qam.real, noisy_16qam.imag, alpha=0.3, s=5)
    for point in qam16_points:
        axes[1, i].plot(point.real, point.imag, 'r*', markersize=10)
    axes[1, i].set_title(f'16-QAM, SNR = {snr} dB')
    axes[1, i].set_xlim(-2, 2)
    axes[1, i].set_ylim(-2, 2)
    axes[1, i].grid(True, alpha=0.3)
    axes[1, i].set_aspect('equal')

axes[0, 0].set_ylabel('Quadrature (Q)')
axes[1, 0].set_ylabel('Quadrature (Q)')
for ax in axes[1, :]:
    ax.set_xlabel('In-Phase (I)')

plt.tight_layout()
plt.show()

### Question 3.1 (10 points)

a) At what SNR do the QPSK constellation points start to overlap?

b) At what SNR do the 16-QAM constellation points start to overlap?

c) Why does 16-QAM require higher SNR than QPSK for reliable detection?

**Your Answer:**

*[Write your answer here]*

## Part 4: Error Vector Magnitude (EVM) (25 points)

EVM measures the deviation of received symbols from ideal constellation points.

$$\text{EVM} = \sqrt{\frac{\frac{1}{N}\sum|\text{received} - \text{ideal}|^2}{P_{\text{avg}}}} \times 100\%$$

### Task 4.1: Calculate EVM

In [ ]:
def calculate_evm(transmitted, received):
    """Calculate EVM in percent."""
    error_vectors = received - transmitted
    error_power = np.mean(np.abs(error_vectors)**2)
    signal_power = np.mean(np.abs(transmitted)**2)
    evm = np.sqrt(error_power / signal_power) * 100
    return evm

# TODO: Calculate EVM for different SNR values
snr_range = np.arange(5, 35, 2)
evm_qpsk = []
evm_16qam = []

for snr in snr_range:
    # QPSK
    noisy = add_awgn(symbols, snr)
    evm_qpsk.append(calculate_evm(symbols, noisy))
    
    # 16-QAM
    noisy = add_awgn(symbols_16qam, snr)
    evm_16qam.append(calculate_evm(symbols_16qam, noisy))

# Plot EVM vs SNR
plt.figure(figsize=(10, 6))
plt.semilogy(snr_range, evm_qpsk, 'b-o', linewidth=2, label='QPSK')
plt.semilogy(snr_range, evm_16qam, 'r-s', linewidth=2, label='16-QAM')

# Add typical WiFi/LTE requirements
plt.axhline(y=8, color='g', linestyle='--', alpha=0.7, label='WiFi 64-QAM limit (8%)')
plt.axhline(y=17.5, color='orange', linestyle='--', alpha=0.7, label='LTE QPSK limit (17.5%)')

plt.xlabel('SNR (dB)')
plt.ylabel('EVM (%)')
plt.title('EVM vs SNR')
plt.legend()
plt.grid(True, alpha=0.3, which='both')
plt.ylim(1, 100)
plt.show()

# Print some values
print("EVM at specific SNR values:")
print(f"{'SNR (dB)':<12} {'QPSK EVM':<15} {'16-QAM EVM':<15}")
for i in [0, 5, 10, 15]:
    print(f"{snr_range[i]:<12} {evm_qpsk[i]:<15.2f} {evm_16qam[i]:<15.2f}")

### Task 4.2: Theoretical EVM vs SNR Relationship

In [ ]:
# TODO: Derive and plot theoretical EVM
# For AWGN: EVM = 100 / sqrt(SNR_linear)
snr_linear = 10**(snr_range / 10)
evm_theoretical = 100 / np.sqrt(snr_linear)

plt.figure(figsize=(10, 6))
plt.semilogy(snr_range, evm_qpsk, 'b-o', linewidth=2, label='QPSK (measured)')
plt.semilogy(snr_range, evm_theoretical, 'k--', linewidth=2, label='Theoretical')
plt.xlabel('SNR (dB)')
plt.ylabel('EVM (%)')
plt.title('EVM vs SNR: Measured vs Theoretical')
plt.legend()
plt.grid(True, alpha=0.3, which='both')
plt.show()

# EVM to SNR conversion
print("\nEVM to SNR conversion:")
print("EVM = 100 / sqrt(SNR_linear)")
print("SNR_dB = -20 * log10(EVM/100)")
print("\nExample: EVM = 10% corresponds to SNR = 20 dB")

### Question 4.1 (15 points)

a) What is the relationship between EVM and SNR for AWGN channels?

b) At what SNR does EVM = 10%?

c) Why do WiFi and cellular standards specify EVM limits instead of (or in addition to) BER?

d) Based on the plot, what SNR is needed for 16-QAM to meet the WiFi 64-QAM EVM limit of 8%?

**Your Answer:**

*[Write your answer here]*

## Part 5: Eye Diagram (10 points)

### Task 5.1: Generate I/Q Eye Diagrams

In [ ]:
# Generate QPSK signal with pulse shaping (simplified)
samples_per_symbol = 8
num_symbols = 200

# Generate random QPSK symbols
bits = np.random.randint(0, 2, num_symbols * 2)
symbols_eye = []
for i in range(0, len(bits), 2):
    bit_pair = (bits[i], bits[i+1])
    symbols_eye.append(qpsk_map[bit_pair])
symbols_eye = np.array(symbols_eye)

# Upsample (repeat symbols)
signal = np.repeat(symbols_eye, samples_per_symbol)

# Add noise
snr_db = 15
noisy_signal = add_awgn(signal, snr_db)

# Create eye diagram
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Plot overlapping symbol periods for I and Q
for i in range(50, 150):  # Plot 100 symbol periods
    start = i * samples_per_symbol
    end = start + 2 * samples_per_symbol  # 2 symbols per trace
    if end <= len(noisy_signal):
        t = np.arange(2 * samples_per_symbol)
        axes[0].plot(t, noisy_signal[start:end].real, 'b-', alpha=0.1)
        axes[1].plot(t, noisy_signal[start:end].imag, 'r-', alpha=0.1)

axes[0].set_xlabel('Sample')
axes[0].set_ylabel('Amplitude')
axes[0].set_title(f'I-Channel Eye Diagram (SNR = {snr_db} dB)')
axes[0].grid(True, alpha=0.3)

axes[1].set_xlabel('Sample')
axes[1].set_ylabel('Amplitude')
axes[1].set_title(f'Q-Channel Eye Diagram (SNR = {snr_db} dB)')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

### Question 5.1 (10 points)

a) What does the "eye opening" in the eye diagram represent?

b) How would the eye diagram change if you increased the noise (lower SNR)?

**Your Answer:**

*[Write your answer here]*

---

## Lab Summary

### Key Concepts

1. **QPSK**: 4 phases, 2 bits/symbol, robust to noise
2. **16-QAM**: 16 points, 4 bits/symbol, higher spectral efficiency
3. **EVM**: Measures modulation quality, related to SNR
4. **Eye Diagram**: Visual tool for signal quality assessment
5. **Trade-off**: Higher-order modulation = more bits but requires better SNR

### Submission Checklist

- [ ] All TODO cells completed
- [ ] All questions answered
- [ ] All cells executed
- [ ] Notebook saved

---

*Submit to Brightspace by due date.*